In [3]:
# !pip install chromadb

import chromadb

In [4]:
# Initialize ChromaDB client (can be in-memory or persistent)
# client = chromadb.Client() # In-memory client

client = chromadb.PersistentClient(path="./github-trending-repositories") # Persistent client


In [5]:
collection_name = "repositories"

# Get or create a collection
# If collection exists with wrong dimension, delete and recreate it
try:
    collection = client.get_collection(name=collection_name)
    # Check if dimension matches (text-embedding-3-small produces 384 dimensions)
    # If you need 1536 dimensions, use text-embedding-3-large or specify dimension
    print(f"Collection '{collection_name}' already exists")
    print(f"Collection metadata: {collection.metadata}")
except:
    # Create collection - specify dimension if needed
    # Option 1: Use 384 dimensions (text-embedding-3-small default)
    collection = client.create_collection(
        name=collection_name,
        metadata={"description": "GitHub trending repositories"}
    )
    print(f"Created new collection '{collection_name}'")

# If you need to reset the collection (delete and recreate with correct dimension):
# Uncomment the following lines:
# try:
#     client.delete_collection(name=collection_name)
#     print(f"Deleted existing collection '{collection_name}'")
# except:
#     pass
# collection = client.create_collection(
#     name=collection_name,
#     metadata={"description": "GitHub trending repositories"}
# )


In [ ]:
# OPTIONAL: Reset collection if dimension mismatch occurs
# Uncomment the code below to delete and recreate the collection
# This will delete all existing data in the collection!

# try:
#     client.delete_collection(name=collection_name)
#     print(f"Deleted collection '{collection_name}'")
# except Exception as e:
#     print(f"Error deleting collection: {e}")

# # Recreate with correct dimension
# collection = client.create_collection(
#     name=collection_name,
#     metadata={"description": "GitHub trending repositories"}
# )
# print(f"Created new collection '{collection_name}'")


In [6]:
import pandas as pd
import glob

# load all csv files from docs folder
one_month_file = glob.glob("../docs/2025-11-09.csv")

# read all csv files
df = pd.concat(map(pd.read_csv, one_month_file), ignore_index=True)

In [ ]:
print(f"Number of rows: {len(df)}")
print(f"Number of columns: {len(df.columns)}")
print('-'*100)

# print(df.head())
print(f"Showing first row:")
print(df.iloc[0])

Number of rows: 20
Number of columns: 21
----------------------------------------------------------------------------------------------------
Showing first row:
full_name                                               Snapchat/Valdi
stargazers_count                                                  2250
owner.login                                                   Snapchat
owner.avatar_url     https://avatars.githubusercontent.com/u/405052...
description          Valdi is a cross-platform UI framework that de...
topics                 ["android","cross-platform","ios","typescript"]
html_url                             https://github.com/Snapchat/Valdi
created_at                                        2025-11-06T17:33:28Z
updated_at                                        2025-11-09T02:29:17Z
pushed_at                                         2025-11-09T00:24:08Z
git_url                            git://github.com/Snapchat/Valdi.git
ssh_url                              git@github.com:Snapch

In [17]:
# !pip install openai
import openai

In [25]:
import numpy as np

def get_embedding(text, dimensions=384):
    """Get embedding from OpenAI and convert to list of native Python floats.
    
    Args:
        text: Text to embed
        dimensions: Embedding dimension (384 for small, 1536 for large)
    """
    # Use text-embedding-3-small for 384 dimensions (default)
    # Use text-embedding-3-large for 1536 dimensions
    # Or use text-embedding-3-small with dimensions parameter for custom sizes
    if dimensions == 1536:
        # Option 1: Use large model (produces 1536 dimensions)
        response = openai.embeddings.create(
            input=text,
            model="text-embedding-3-large"
        )
    elif dimensions == 384:
        # Use small model (produces 384 dimensions)
        response = openai.embeddings.create(
            input=text,
            model="text-embedding-3-small"
        )
    else:
        # Use small model with custom dimensions
        response = openai.embeddings.create(
            input=text,
            model="text-embedding-3-small",
            dimensions=dimensions
        )
    
    embedding = response.data[0].embedding
    # Convert numpy types to native Python types for JSON serialization
    if isinstance(embedding, np.ndarray):
        return [float(x) for x in embedding]
    elif isinstance(embedding, list):
        return [float(x) if isinstance(x, (np.floating, np.float64, np.float32)) else x for x in embedding]
    return embedding


In [22]:
get_embedding(df.iloc[0]["description"])

[-0.013116688467562199,
 0.011234354227781296,
 0.0035360045731067657,
 0.026816634461283684,
 0.021938472986221313,
 -0.04021832346916199,
 -0.04294903576374054,
 -0.010949352756142616,
 0.024112436920404434,
 -0.009815975092351437,
 -0.018889620900154114,
 0.0028599549550563097,
 -0.020069394260644913,
 -0.004278333391994238,
 0.04281647503376007,
 0.0007862390484660864,
 -0.023714760318398476,
 -0.04578579217195511,
 -0.016755426302552223,
 0.059386320412158966,
 0.030037812888622284,
 -0.037089936435222626,
 0.007178056053817272,
 0.04069553688168526,
 0.01743147522211075,
 -0.029109900817275047,
 -0.03348433971405029,
 0.022110799327492714,
 -0.015496117994189262,
 -0.049285341054201126,
 0.034730393439531326,
 -0.031204329803586006,
 0.04721742495894432,
 -0.05180395767092705,
 0.03178758919239044,
 0.009670160710811615,
 0.02090451493859291,
 0.0055873510427773,
 -0.06384029239416122,
 0.015522629022598267,
 -0.08664039522409439,
 -0.020241720601916313,
 0.03788528963923454,
 0.

In [26]:
df["embedding"] = df["description"].apply(lambda x: get_embedding(x, dimensions=1536))


In [65]:
# only pick the description no na
df = df[df["description"].notna()]

In [66]:
print(len(df))

17


In [67]:
for embedding in df.embedding.tolist():
    print(f"embedding type: {type(embedding)}, value: {embedding}, length: {len(embedding)}")

embedding type: <class 'list'>, value: [-0.013116688467562199, 0.011234354227781296, 0.0035360045731067657, 0.026816634461283684, 0.021938472986221313, -0.04021832346916199, -0.04294903576374054, -0.010949352756142616, 0.024112436920404434, -0.009815975092351437, -0.018889620900154114, 0.0028599549550563097, -0.020069394260644913, -0.004278333391994238, 0.04281647503376007, 0.0007862390484660864, -0.023714760318398476, -0.04578579217195511, -0.016755426302552223, 0.059386320412158966, 0.030037812888622284, -0.037089936435222626, 0.007178056053817272, 0.04069553688168526, 0.01743147522211075, -0.029109900817275047, -0.03348433971405029, 0.022110799327492714, -0.015496117994189262, -0.049285341054201126, 0.034730393439531326, -0.031204329803586006, 0.04721742495894432, -0.05180395767092705, 0.03178758919239044, 0.009670160710811615, 0.02090451493859291, 0.0055873510427773, -0.06384029239416122, 0.015522629022598267, -0.08664039522409439, -0.020241720601916313, 0.03788528963923454, 0.0177

In [68]:
for topic in df.topics.tolist():
    print(f"type: {type(topic)}, value: {topic}, length: {len(topic)}")

type: <class 'str'>, value: ["android","cross-platform","ios","typescript"], length: 47
type: <class 'str'>, value: [], length: 2
type: <class 'str'>, value: [], length: 2
type: <class 'str'>, value: ["bitcoin-transaction","blockchain","blockchain-wallet","ethereum","ethereum-contract","ethereum-dapp","ethereum-wallet","indicator-analysis","ledger-wallet","solana","solana-memecoin-sniper-bot","solana-rug","solana-token-launcher","solanamemecoines","web3","web3-dapp","web3js"], length: 280
type: <class 'str'>, value: [], length: 2
type: <class 'str'>, value: [], length: 2
type: <class 'str'>, value: ["ai","artificial-intelligence","sora-video","sora2-ai","watermark-remover"], length: 76
type: <class 'str'>, value: [], length: 2
type: <class 'str'>, value: ["twitter-api"], length: 15
type: <class 'str'>, value: ["cli","mcp","ts-api"], length: 22
type: <class 'str'>, value: [], length: 2
type: <class 'str'>, value: ["cve-2025-59287"], length: 18
type: <class 'str'>, value: [], length: 2
t

In [69]:
import json
def parse_topics(topics_str):
    """Convert string representation of array to Python list."""
    if pd.isna(topics_str) or topics_str is None:
        return []
    if isinstance(topics_str, list):
        return topics_str
    if isinstance(topics_str, str):
        topics_str = topics_str.strip()
        if not topics_str or topics_str == '[]':
            return []
        # Try json.loads() first
        try:
            return json.loads(topics_str)
        except json.JSONDecodeError:
            # Fallback to ast.literal_eval()
            try:
                import ast
                return ast.literal_eval(topics_str)
            except (ValueError, SyntaxError):
                return [topics_str] if topics_str else []
    return []

# Create metadatas as list of dictionaries
metadatas = []
for topic in df.topics.tolist():
    metadatas.append(json.loads(topic))

print(len(metadatas))
print(metadatas)

17
[['android', 'cross-platform', 'ios', 'typescript'], [], [], ['bitcoin-transaction', 'blockchain', 'blockchain-wallet', 'ethereum', 'ethereum-contract', 'ethereum-dapp', 'ethereum-wallet', 'indicator-analysis', 'ledger-wallet', 'solana', 'solana-memecoin-sniper-bot', 'solana-rug', 'solana-token-launcher', 'solanamemecoines', 'web3', 'web3-dapp', 'web3js'], [], [], ['ai', 'artificial-intelligence', 'sora-video', 'sora2-ai', 'watermark-remover'], [], ['twitter-api'], ['cli', 'mcp', 'ts-api'], [], ['cve-2025-59287'], [], ['docker', 'evernote', 'fastapi', 'free', 'hosted', 'knowledge-base', 'markdown', 'note', 'note-taking', 'notion', 'obsidian', 'open-source', 'privacy', 'python', 'self-hosted'], ['agentic-ai', 'modular', 'scaleable'], [], []]


In [70]:
print(df["description"].tolist())

['Valdi is a cross-platform UI framework that delivers native performance without sacrificing developer velocity.', 'pg_lake: Postgres with Iceberg and data lake access', 'An open-source framework for provably private AI inference', 'Open-source Solana dApp template with Anchor program, TypeScript client, and Next.js UI.', 'Breathe fresh life into your bricked Nest Gen 1 & 2, now with 100% less evil!', '自动登录netlib.re网站保活域名', 'Sora 2 AI generated videos gentle watermark remover', '一个将ACE(Augment Context Engine) 做成MCP的项目', 'a powerful twitter video downloader and twitter marketing online tool. repost any tweet auto translation.', 'Call MCPs via TypeScript, masquerading as simple TypeScript API. Or package them as cli.', 'SilentButDeadly is a network communication blocker specifically designed to neutralize EDR/AV software by preventing their cloud connectivity using Windows Filtering Platform (WFP). This version focuses solely on network isolation without process termination.', 'WSUS vul

In [ ]:
# Add documents with embeddings and metadata
# First, ensure we only process rows with valid embeddings
import numpy as np
import json
import ast

# Filter out rows with None/NA embeddings
df_valid = df[df["embedding"].notna()].copy()
print(f"Adding {len(df_valid)} documents with valid embeddings to ChromaDB")

# Convert embeddings to ensure they're JSON-serializable
embeddings_list = df_valid["embedding"].tolist()
embeddings_list = [
    [float(x) for x in emb] if isinstance(emb, (list, np.ndarray)) else emb
    for emb in embeddings_list
]

# Parse topics function - convert string array to actual array
def parse_topics(topics_str):
    """Convert string representation of array to Python list."""
    if pd.isna(topics_str) or topics_str is None:
        return []
    if isinstance(topics_str, list):
        return topics_str
    if isinstance(topics_str, str):
        topics_str = topics_str.strip()
        if not topics_str or topics_str == '[]':
            return []
        try:
            return json.loads(topics_str)
        except json.JSONDecodeError:
            try:
                return ast.literal_eval(topics_str)
            except (ValueError, SyntaxError):
                return [topics_str] if topics_str else []
    return []

# Convert metadata to ensure JSON serialization compatibility
def convert_to_serializable(obj):
    """Convert numpy types to native Python types."""
    if isinstance(obj, (np.integer, np.int64, np.int32)):
        return int(obj)
    elif isinstance(obj, (np.floating, np.float64, np.float32)):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, (list, tuple)):
        return [convert_to_serializable(item) for item in obj]
    elif isinstance(obj, dict):
        return {key: convert_to_serializable(value) for key, value in obj.items()}
    elif isinstance(obj, float):
        if np.isinf(obj) or np.isnan(obj):
            return None
        return obj
    return obj

# Create metadatas as a list of dictionaries (ChromaDB requirement)
# Each metadata must be a dictionary, not a string or list
metadatas = []
for idx, row in df_valid.iterrows():
    metadata = {
        "full_name": str(row.get("full_name", "")),
        "stargazers_count": int(row.get("stargazers_count", 0)) if pd.notna(row.get("stargazers_count")) else 0,
        "language": str(row.get("language", "")) if pd.notna(row.get("language")) else "",
        "forks_count": int(row.get("forks_count", 0)) if pd.notna(row.get("forks_count")) else 0,
        "topics": parse_topics(row.get("topics", [])),  # Parse topics from string to list
        "html_url": str(row.get("html_url", ""))
    }
    # Convert all values to serializable format
    metadata = convert_to_serializable(metadata)
    metadatas.append(metadata)

# Verify metadatas structure
print(f"\nMetadatas type: {type(metadatas)}")
print(f"Number of metadatas: {len(metadatas)}")
if metadatas:
    print(f"First metadata type: {type(metadatas[0])}")
    print(f"First metadata: {metadatas[0]}")

# Add to ChromaDB
collection.add(
    documents=df_valid["description"].tolist(),
    metadatas=metadatas,  # List of dictionaries
    ids=[str(identifier) for identifier in df_valid['full_name'].tolist()],
    embeddings=embeddings_list
)

print(f"\nSuccessfully added {len(df_valid)} documents to ChromaDB collection")


AttributeError: 'list' object has no attribute 'items' in add.